Funcoes utilizadas
Criar uma lista de comprimentos de strings em cada célula da coluna especificada
comprimentos = [len(str(celula)) for celula in df.iloc[: ,4]]
Encontrar o índice da célula com o maior número de caracteres
indice_maior = comprimentos.index(max(comprimentos))
Extrair a célula com o maior número de caracteres
celula_maior = df.iloc[indice_maior][4] df.iloc[: , 4][indice_maior] df.iloc[: , 4].value_counts() celula_maior df.iloc[:,0]
index = df.columns.get_loc(‘abcz’)
index df

IDANIMAL = CHAVE UNICA
MP = CHAVE UNICA -  Mes parto
AP = CHAVE UNICA - -Ano parto

LTOT = lactacao total
L365 = lactacao em ate 365 dias
L305 = lactacao em ate 305 dias
DE = Data de Secagem
ME = Mes de secagem
AE = ANO de Secagem
caso o LTOT, L365 e L305, DE, ME e AE sejam iguais ao arquivo old, pode ser descartado

In [ ]:
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')

## conectando no banco de dados

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

## Retornando toda tabela genealogia_embrapa e genealogia_abcz e tabela criadores do banco de dados para dataframes

In [ ]:
sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
sql1 = "SELECT * FROM genealogia_abcz"
sql2 = "SELECT * FROM criadores"
sql3 = "SELECT * FROM producao_embrapa"
df_genealogia_embrapa = pd.read_sql(sql, mydb)
df_genealogia_embrapa=df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = pd.read_sql(sql1, mydb)
df_criadores = pd.read_sql(sql2, mydb)
df_producao = pd.read_sql(sql3, mydb)


## Lendo planilhas

In [ ]:
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [ ]:
#pedigree

In [ ]:
#dados_2022

In [ ]:
#dados_2023

In [ ]:
#df_genealogia_embrapa

In [ ]:
#df_genealogia_abcz

In [ ]:
#df_criadores

In [ ]:
#df_producao

## Criando um dataframe que é a junção dos dois dataframes, dados_2022 e dados_2023, pelas celulas que possuem os mesmos valores nas colunas IDANIMAL, AP, MP, L305, LTOT, L365, ME e AE.

In [ ]:
df_dados_sem_atualizacao_entre_2022_2023 = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop(df_dados_sem_atualizacao_entre_2022_2023.filter(regex='_drop$').columns, axis=1)
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop_duplicates()
#df_dados_sem_atualizacao_entre_2022_2023

## Criando um dataframe com somente os novos dados inseridos em 2023
Esses são os dados que precisam ser inseridos no banco de dados

In [ ]:
# faz o merge entre os dois dataframes, o dados com os dados repetidos
df_merge = pd.merge(dados_2023, df_dados_sem_atualizacao_entre_2022_2023, how='left', indicator=True, suffixes=('', '_drop'))

# filtra somente as linhas que estão no dataframe da esquerda
df_dados_novos_para_insercao_no_banco_de_dados = df_merge[df_merge['_merge'] == 'left_only']

# remove a coluna "_merge"
df_dados_novos_para_insercao_no_banco_de_dados.drop('_merge', axis=1, inplace=True)

df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_merge.filter(regex='_drop$').columns, axis=1)
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando coluna GS, oriunda do dataframe Pedigree, no dataframe df_dados_novos_para_insercao_no_banco_de_dados
inclusão necessária para localizar o animal

In [ ]:
df_temporaria = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria
df_dados_novos_para_insercao_no_banco_de_dados

nome_antigo = 'SEXO'
nome_novo = 'sexo'
indice_coluna = df_dados_novos_para_insercao_no_banco_de_dados.columns.get_loc(nome_antigo)
df_dados_novos_para_insercao_no_banco_de_dados.rename(columns={nome_antigo: nome_novo}, inplace=True)
#df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela com somente as linhas que possuem gs = 0 no dataframe df_dados_novos_para_insercao_no_banco_de_dados

In [ ]:
sub_tabela_gs_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['gs'] == 0]
sub_tabela_gs_0

# Adicionando coluna cod_prod, cod_pai e cod_mae, oriundos do dataframe df_genealogia_abcz, na subtabela gs_0 dos animais que sao gs=0

In [ ]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
df_temporaria1 = pd.merge(sub_tabela_gs_0, df_genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
sub_tabela_gs_0=df_temporaria1
sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()
sub_tabela_gs_0

# Adicionando codprod, cod_pai, cod_mae no dataframe df_dados_novos_para_insercao_no_banco_de_dados a partir do dataframe sub_tabela_gs_0

In [ ]:
df_temporaria4 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria4
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados



# Criando subtabela com os animais em que o gs da mae = 11

In [ ]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSMAE'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_mae_0

# Pegando o cod_prod das maes no dataframe genealogia_abcz

In [ ]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
sub_tabela_cod_mae_0 = pd.merge(sub_tabela_cod_mae_0, df_genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop(sub_tabela_cod_mae_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_mae_0


# Adicionando o cod_prod_mae_gen das maes no dataframe df_dados_novos

In [ ]:
df_temporaria5 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_mae_0[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria4
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria5 = df_temporaria5.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria5
df_dados_novos_para_insercao_no_banco_de_dados


# Criando subtabela em que o gs do pai = 11 e que os 3 primeiros caracteres do rgpai sao letras

In [ ]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSPAI'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_pai_0
# selecionar apenas as linhas que começam com letras
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando o cod_prod dos pais no dataframe genealogia_abcz

In [ ]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
sub_tabela_cod_pai_0 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop(sub_tabela_cod_pai_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_pai_0

# Adicionando o cod_prod_pai_gen dos pais no dataframe df_dados_novos

In [ ]:
df_temporaria6 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria6 = df_temporaria6.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria6
df_dados_novos_para_insercao_no_banco_de_dados

# Pegando valores das colunas cod_prod_pai_gen e cod_prod_mae_gen e atualizando colunas  cod_pai e cod_mae no DF_DADOS_NOVOS

In [ ]:
df_temporaria6 = df_temporaria5
df_dados_novos_para_insercao_no_banco_de_dados['cod_pai'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_pai_gen'], inplace=True)
df_dados_novos_para_insercao_no_banco_de_dados['cod_mae'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_mae_gen'], inplace=True)
#Dando drop nas colunas antigas
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_pai_gen')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_mae_gen')
df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela em que o gs do pai = 11 mas os 3 primeiros caracteres do rgpai nao sao letras

In [ ]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSPAI'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_pai_0
# selecionar apenas as linhas que NÃO começam com letras
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[~sub_tabela_cod_pai_0['nome'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando cod_prod_pai no genealogia_abcz dos animais em que o registro nao comeca com 3 letras

In [ ]:
#criando coluna com os 10 caracteres do nome
sub_tabela_cod_pai_0['nome10caracteres'] = sub_tabela_cod_pai_0['nome'].str.slice(start=0, stop=11)
df_genealogia_abcz['nome10caracteres'] = df_genealogia_abcz['nomea'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
sub_tabela_cod_pai_0['nome10caracteres'] = sub_tabela_cod_pai_0['nome10caracteres'].str.replace('\s+', '', regex=True)
df_genealogia_abcz['nome10caracteres'] = sub_tabela_cod_pai_0['nome10caracteres'].str.replace('\s+', '', regex=True)

sub_tabela_cod_pai_0 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI', 'nome10caracteres'], right_on=['RGVACA', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop(sub_tabela_cod_pai_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_pai_0

# Adicionando o cod_prod_pai_gen dos pais no dataframe df_dados_novos
obs: dos REGPAI que nao comecam com 3 caracteres

In [ ]:
df_temporaria7 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria7 = df_temporaria7.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria7
df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela em que o gs da mae = 11 mas os 3 primeiros caracteres do rgmae nao sao letras

In [ ]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSMAE'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_mae_0
# selecionar apenas as linhas que NÃO começam com letras
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[~sub_tabela_cod_mae_0['nome'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando cod_prod_mae no genealogia abcz dos animais em que o registro nao comeca com 3 letras

In [ ]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
sub_tabela_cod_mae_0 = pd.merge(sub_tabela_cod_mae_0, df_genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE', 'nome10caracteres'], right_on=['RGVACA', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop(sub_tabela_cod_mae_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_mae_0


# Adicionando cod_prod_mae_gen no df_dados_novos
Obs: maes com registro iniciando com menos de 3 caracteres

In [ ]:
df_temporaria8 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria8 = df_temporaria8.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria8
df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando reb no dataframe df_dados_novos_para_insercao_no_banco_de_dados
Obs: Cod_gil esta na tabela de criadores

In [ ]:
df_criadores = df_criadores.rename(columns={'codgiro':'reb_giro'})

df_dados_novos_para_insercao_no_banco_de_dados = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, df_criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()


In [ ]:
#Pegando os dados das colunas

In [ ]:
#verifica_reb = df_criadores.groupby('reb_giro')['reb'].nunique()
#verifica_reb

## Calculando a diferença entre a data dos novos partos com as ultimas datas presentes na tabela de dados_2022 para validar se ha alguma inconsistência

obs: Caso a diferença seja menor que 250 dias é provável que haja alguma falha.

In [ ]:

df_diferenca_entre_partos_novos_e_do_banco = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on=['IDANIMAL'], suffixes=('', '_drop'))
df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop_duplicates()
#df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop(df_diferenca_entre_partos_novos_e_do_banco.filter(regex='_drop$').columns, axis=1)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].astype(str)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].apply(pd.to_datetime)
df_diferenca_entre_partos_novos_e_do_banco['diferenca_em_dias'] = (df_diferenca_entre_partos_novos_e_do_banco['parto'] - df_diferenca_entre_partos_novos_e_do_banco['parto_drop']).dt.days
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop', 'diferenca_em_dias']]

## Pegando da tabela de animais que serão inseridos no banco aqueles que ja possuem IDANIMAL cadastrado
Comparado ao dataframe  dados_2022

In [ ]:
df_animais_com_ID = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on =['IDANIMAL'], suffixes=('', '_drop'))
df_animais_com_ID = df_animais_com_ID.drop(df_animais_com_ID.filter(regex='_drop$').columns, axis=1)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID

## Pegando da tabela de animais que serão inseridos no banco, todos aqueles que não possuem IDANIMAL cadastrado
comparado ao dataframe dados_2022

In [ ]:
df_animais_sem_id = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on='IDANIMAL', how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['_merge'] == 'left_only'].drop(columns='_merge')
df_animais_sem_id

In [ ]:
#dados_2022[dados_2022['IDANIMAL'] == 1631099]
#df_verificando_animais_com_id_na_genealogia_embrapa['IDANIMAL'].value_counts()
#verifica_reb = df_verificando_animais_com_id_na_genealogia_embrapa.groupby('IDANIMAL')['id_produto'].nunique()
#verifica_reb

# Adicionando a coluna id_produto ao dataframe df_animais_com_ID

In [ ]:
df_genealogia_embrapa['IDANIMAL'] = df_genealogia_embrapa['IDANIMAL'].astype('int')
df_animais_com_ID = pd.merge(df_animais_com_ID, df_genealogia_embrapa[['IDANIMAL', 'id_produto']], on='IDANIMAL', how='left', indicator=True)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID = df_animais_com_ID.drop(columns='_merge')
df_animais_com_ID

## Lista de animais do dataframe sem ID em relacao a dados_2022 e que possuem ID_animal no dataframe da genealogia_Embrapa
Obs: Tem que ter ID_produto

In [ ]:
df_animais_sem_id = df_animais_sem_id.add_suffix('_esq')
df_genealogia_embrapa = df_genealogia_embrapa.add_suffix('_dir')

df_animais_sem_id_2022_que_possuem_id_no_banco = pd.merge(df_animais_sem_id, df_genealogia_embrapa, left_on=['IDANIMAL_esq'], right_on=['IDANIMAL_dir'], how='left', indicator=True)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop_duplicates()
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco[df_animais_sem_id_2022_que_possuem_id_no_banco['_merge'] == 'both']

# dropa as colunas que não são necessárias
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_dir').columns.difference(['id_produto_dir']), axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco

# Criando dataframe com todos os animais que possuem ID encontrado

In [ ]:
# Juntando os dataframes
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_sem_id_2022_que_possuem_id_no_banco])
df_animais_com_ID

# Atualizando lista de animais sem ID

In [ ]:
#Tirando o sufixo da tabela
df_animais_sem_id = df_animais_sem_id.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
# Imprimir o resultado
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto'].isnull()]
df_animais_sem_id

# Fazendo busca no dataframe producao para encontrar id_produto dos animais sem id
conferir manualmente se entram ou não no banco

In [ ]:
#Criando coluna com o ano de parto e mes parto no dataframe df_producao
df_producao['parto'] = df_producao['parto'].astype(str)
df_producao['AP'] = df_producao['parto'].str.slice(start=0, stop=4)
df_producao['MP'] = df_producao['parto'].str.slice(start=5, stop=7)
#Pegando os 10 primeiros caracteres de nome dos dataframes aniamis sem id e producao
df_producao['nome10caracteres'] = df_producao['nome'].str.slice(start=0, stop=11)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nomea'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
df_producao['nome10caracteres'] = df_producao['nome10caracteres'].str.replace('\s+', '', regex=True)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nome10caracteres'].str.replace('\s+', '', regex=True)

df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_producao = df_producao.drop(df_producao.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns='_merge')

df_producao['AP'] = df_producao['AP'].astype(int)
df_producao['MP'] = df_producao['MP'].astype(int)
df_animais_sem_id['AP'] = df_animais_sem_id['AP'].astype(int)
df_animais_sem_id['MP'] = df_animais_sem_id['MP'].astype(int)
df_animais_com_id_na_producao = pd.merge(df_animais_sem_id, df_producao[['reb','AP', 'MP', 'nome10caracteres','id_produto']], on=['reb', 'AP', 'MP', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_producao = df_animais_com_id_na_producao[df_animais_com_id_na_producao['_merge'] == 'both']
df_animais_com_id_na_producao = df_animais_com_id_na_producao.drop(columns={'_merge', 'id_produto'})
df_animais_com_id_na_producao = df_animais_com_id_na_producao.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_producao

# Atualizando animais com id com os novos ids_produtos encontrados no dataframe de producao

In [ ]:
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_com_id_na_producao])
df_animais_com_ID

# Atualizando lista de animais sem id

In [ ]:
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto_drop'].isnull()]
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id

# Fazendo a busca na tabela genealogia_embrapa

In [ ]:
#Retirando sufixo tabela genealogia
df_genealogia_embrapa = df_genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
#Pegando ano nascimento nas colunas
df_genealogia_embrapa['nasc_est'] = df_genealogia_embrapa['nasc_est'].astype(str)
df_genealogia_embrapa['AN'] = df_genealogia_embrapa['nasc_est'].str.slice(start=0, stop=4)
df_animais_sem_id['nasc'] = df_animais_sem_id['nasc'].astype(str)
df_animais_sem_id['AN'] = df_animais_sem_id['nasc'].str.slice(start=0, stop=4)
#Criando coluna com os 10 caracteres do nome na genealogia_embrapa
df_genealogia_embrapa['nome10caracteres'] = df_genealogia_embrapa['nome'].str.slice(start=0, stop=11)
#Convertendo coluna ano e mes para tipo inteiro
#df_genealogia_embrapa['AN'] = df_genealogia_embrapa['AN'].astype(int)
#Limpando tabelas
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_genealogia_embrapa = df_genealogia_embrapa.drop(df_genealogia_embrapa.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns={'_merge'})
#criando dataframe com os ids_encontrados
df_animais_com_id_na_genealogia = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs','RGMAE', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia[df_animais_com_id_na_genealogia['_merge'] == 'both']
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(columns={'_merge', 'id_produto', 'registro_mae', 'registro_pai'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(df_animais_com_id_na_genealogia.filter(regex='_drop$').columns, axis=1)
df_animais_com_id_na_genealogia

In [ ]:
#Conferência manual dos matchs com o dataframe da genealogia

In [ ]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

# Adicionar o que ta escrito no quadro

In [ ]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

In [ ]:
df_animais_com_id_na_genealogia2 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['id_produto','cod_prod']], left_on=['cod_prod'], right_on=['cod_prod'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia2 = df_animais_com_id_na_genealogia2[df_animais_com_id_na_genealogia2['_merge'] == 'both']
df_animais_com_id_na_genealogia2